In [1]:
import tools.sleeper_read as sr
from tools.sql_cnx import sql_cnx
import pandas as pd
import numpy as np

import importlib

### Build overall player table

Sleeper has player data that updates less often and is not unique to leagues

Pushes data to a data warehouse, in this case Planetscale using sql_cnx, which is a custom SQLAlchemy wrapper made to interface with Planetscale

In [2]:
#build player table and upload it to planetscale

#uncomment below to build:

# import requests

# player_table = requests.get('https://api.sleeper.app/v1/players/nfl')
# players = pd.DataFrame(player_table.json()).transpose()
# players = players.reset_index().drop('index',axis = 1)
# players = players.infer_objects()

# for n,i in enumerate(players.dtypes):
#     if i == 'object':
#         players[str(players.columns[n])] = players[str(players.columns[n])].astype(str)

# instance = sql_cnx(env_path='init/.env')

# instance = sql_cnx(env_path='init/.env')
# conn = instance.cnx
# players.to_sql(name='sleeper_players',con=conn,if_exists='replace')
# instance.cleanup()

# instance.cleanup()

instance = sql_cnx(env_path='init/.env')
conn = instance.cnx
x = pd.read_sql('select * from sleeper_players limit 100',con = conn)
instance.cleanup()

x.head()

,index,fantasy_data_id,status,injury_start_date,years_exp,yahoo_id,practice_description,position,hashtag,gsis_id,...,depth_chart_position,injury_status,search_first_name,full_name,birth_date,birth_country,pandascore_id,active,birth_city,fantasy_positions
0,0,18829.0,Inactive,None,0.0,NaN,None,None,#davecaldwell-NFL-FA-0,None,...,None,None,dave,Dave Caldwell,None,None,None,0,None,None
1,1,19705.0,Active,None,6.0,30715.0,None,WR,#gregward-NFL-PHI-84,00-0033733,...,SWR,None,greg,Greg Ward,1995-07-12,None,None,1,None,['WR']
2,2,15244.0,Inactive,None,7.0,26894.0,None,G,#alvinbailey-NFL-FA-78,None,...,None,None,alvin,Alvin Bailey,1991-08-26,None,None,0,None,['OL']
3,3,13170.0,Injured Reserve,None,11.0,24942.0,None,TE,#nilespaul-NFL-FA-0,00-0028093,...,None,None,niles,Niles Paul,1989-08-09,None,None,0,None,['TE']
4,4,19247.0,Active,None,5.0,30668.0,None,OT,#kwaydemiller-NFL-FA-63,None,...,None,None,kwayde,Kwayde Miller,1993-12-11,None,None,1,None,['OL']


### Test Out sleeper_read.py Package

In [3]:
# initialize league read:
# importlib temporary while working on library

importlib.reload(sr)


<module 'tools.sleeper_read' from 'c:\\Users\\samml\\Documents\\PyProj\\Planetscale\\tools\\sleeper_read.py'>

In [4]:
id = 832718546487357440
data = sr.ffData(id)

print(f'''BUILT TABLES: {[attr for attr in dir(data) if attr.endswith('table')]}
FETCH FUNCS: {[attr for attr in dir(data) if attr.startswith(('get'))]} 
RAW FROM FETCH: {[attr for attr in dir(data) if (not attr.startswith(('__','get','_')) and not attr.endswith('table'))]}
CONST: {[attr for attr in dir(data) if (attr.startswith(('_')) and not attr.startswith(('__')))]}''')


BUILT TABLES: ['matchup_table', 'results_and_leg_roster_table', 'users_table']
FETCH FUNCS: ['get_league', 'get_matchups', 'get_nfl_state', 'get_rosters', 'get_transactions', 'get_users'] 
RAW FROM FETCH: ['league', 'matchups', 'nfl', 'rosters', 'transactions', 'users']
CONST: ['_current_week', '_playoff_week', '_season']


In [5]:
data.users_table

,user_id,username,roster_id
0,465633370747957248,LEDRs,1
1,465636493809938432,smlederer,2
2,466716381887328256,bbrbr,3
3,469271575548391424,zgreen1017,4
4,469319841732358144,aleith,5
5,469921663765442560,mattdemichiel,6
6,471147635869216768,Subject2Change,7
7,472159133173280768,nlfish28,9
8,472234548525002752,twtytwty,10
9,728408081750405120,HeMarv,12


In [6]:
data.matchup_table

,week,matchup_id,roster_id_root,user_id_root,username_root,roster_id_challenger,user_id_challenger,username_challenger
1,1,2,1,465633370747957248,LEDRs,10,472234548525002752,twtytwty
2,1,2,10,472234548525002752,twtytwty,1,465633370747957248,LEDRs
5,1,6,2,465636493809938432,smlederer,11,731254033469263872,H1nduHammer
6,1,6,11,731254033469263872,H1nduHammer,2,465636493809938432,smlederer
9,1,3,3,466716381887328256,bbrbr,7,471147635869216768,Subject2Change
...,...,...,...,...,...,...,...,...
326,14,1,8,866029249553944576,Mnorts,4,469271575548391424,zgreen1017
329,14,4,5,469319841732358144,aleith,12,728408081750405120,HeMarv
330,14,4,12,728408081750405120,HeMarv,5,469319841732358144,aleith
333,14,5,6,469921663765442560,mattdemichiel,9,472159133173280768,nlfish28


In [7]:
data.results_and_leg_roster_table

,roster_id,points,players,matchup_id,custom_points,is_starter,user_id,username,week
0,1,111.38,SF,2,None,True,465633370747957248,LEDRs,1
1,1,111.38,8129,2,None,True,465633370747957248,LEDRs,1
2,1,111.38,7525,2,None,True,465633370747957248,LEDRs,1
3,1,111.38,6820,2,None,True,465633370747957248,LEDRs,1
4,1,111.38,5927,2,None,True,465633370747957248,LEDRs,1
...,...,...,...,...,...,...,...,...,...
179,12,107.70,4018,4,None,True,728408081750405120,HeMarv,14
180,12,107.70,2749,4,None,False,728408081750405120,HeMarv,14
181,12,107.70,1817,4,None,True,728408081750405120,HeMarv,14
182,12,107.70,1234,4,None,True,728408081750405120,HeMarv,14
